## 测试一些新的简单估计方法

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sys
sys.path.append('C:/Users/Administrator/Desktop/Repositories/Low-Frequency-Spread-Estimator')
sys.path.append('C:/Users/Handsome Bad Guy/Desktop/Repositories/Low-Frequency-Spread-Estimator')

from SpreadEstimator.SpreadEstimator import SpreadEstimator
from mytools.AutoTester import AutoTester

from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim

from sklearn.linear_model import LinearRegression

In [4]:
se = SpreadEstimator()
univ = ~np.isnan(se.data.data_dic['close'])  # 合法的univ

In [5]:
se.data.data_dic.keys()

dict_keys(['open', 'low', 'high', 'close', 'last_bas', 'last_rel_bas', 'cap', 'volume'])

In [33]:
fmls = [
    'tsmean{div{absv{tspct{close,1}},volume},20}',  # Amihud中用到的一个指标
    'tsmean{div{absv{tspct{close,1}},powv{volume,0.5}},20}',  # 这个对低价股的反向比较明显
    'tsmean{div{high,low},20}',  # 20天平均振幅，对低价股和高价股是反向的
    'neg{tsmean{powv{cap,0.5},20}}',  # 市值越小相对价差越大
    
]

fml =  'neg{tsmean{powv{cap,0.5},20}}'

stats, signal = se.test_factor(fml, corr_type='linear',method='cs', spread_type='relative_spread', back=20)

corr = []
sig = []
est = []
for i in range(20,240):
    sp = np.mean(se.data.spread_dic['vol_wtd_rel_bas'][i-20+1:i+1], axis=0)
    top = (~np.isnan(sp)) & (~np.isnan(signal[i])) & (np.mean(se.data.data_dic['close'][i-19:i],axis=0)>25)
    # print(np.sum(top))
    sig += list(signal[i,top])
    est += list(sp[top])
    corr.append(np.corrcoef(sp[top],signal[i,top])[0,1])
# plt.scatter(est,sig)
print('vol_wtd_rel_bas cs: {:.4f}'.format(np.mean(corr)))
      
corr = []
sig = []
est = []
new_sp = se.data.spread_dic['vol_wtd_rel_bas'].copy()
for i in range(se.data.spread_dic['vol_wtd_rel_bas'].shape[0]):
    if i < 20 - 1:
        continue
    new_sp[i] = np.mean(se.data.spread_dic['vol_wtd_rel_bas'][i-19:i+1], axis=0)
for i in range(new_sp.shape[1]):
    sp = new_sp[:,i]
    top = (~np.isnan(sp)) & (~np.isnan(signal[:,i])) & (se.data.data_dic['close'][:,i] > 25)
    if np.nanmean(se.data.data_dic['close'][:,i]) < 25:
        continue
    sig += list(signal[:,i][top])
    est += list(sp[top])
    corr.append(np.corrcoef(sp[top],signal[:,i][top])[0,1])
# plt.scatter(est,sig)
print('vol_wtd_rel_bas ts: {:.4f}'.format(np.nanmean(corr)))


mean corr: 0.3542, positive_corr_ratio: 1.0000, corr_IR: 5.1405
vol_wtd_rel_bas cs: 0.5361
vol_wtd_rel_bas ts: 0.1293
